# Single Model Assisted Sensitivity Analysis
This notebook focuses on answering the question: how can AutoMATES leverage the information extracted from source code and free text to improve a modeler's ability to visualize input sensitivity in a single model?

In this notebook we will demonstrate AutoMATES ability to analyze the Priestley-Taylor model of Evapotranspiration from the DSSAT codebase.

First, we import a few necessary items.

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
import jupyter_tools as jt

The code for the Priestley-Taylor model as implemented in DSSAT is shown below.

In [ ]:
jt.display_code("../tests/data/program_analysis/PETPT.for")

We first create a GroundedFunctionNetwork object.

In [ ]:
from model_analysis.networks import GroundedFunctionNetwork
G = GroundedFunctionNetwork.from_fortran_file(
    "../tests/data/program_analysis/PETPT.for"
)

We can visualize it as a computational graph...

In [ ]:
jt.display(G.to_AGraph())

...or a variable dependency graph (we also call it a causal analysis graph)

In [ ]:
jt.display(G.CAG_to_AGraph())

Now we are ready to run sensitivity analysis on this GroundedFunctionNetwork (GrFN) object.

### Sampling over the whole range

We first sample across the whole allowed range for each input variable, over five orders of magnitude of sample sizes: $(10^1, 10^2, 10^3, 10^4, 10^5)$.

In [ ]:
from model_analysis.SensitivityModel import SensitivityModel
from model_analysis.visualization import SensitivityVisualizer

# We run the analysis for some orders of magnitude.
sample_sizes = [10**x for x in range(1, 6)]

bounds = {
    'tmax':[-10.0, 50.0],
    'tmin':[0.0, 23.9],
    'srad': [2.45, 27.8],
    'msalb': [0.18, 0.2],
    'xhlai': [0.0, 4.77]
}

SM = SensitivityModel(G, bounds, sample_sizes, 'Sobol', model_name = 'PETPT')

indices = SM.generate_indices()

Below we show a plot of the S1 sensitivity indices for individual variables vs. sample size. We see that as we take more samples, the value of the S1 indices stabilize.

In [ ]:
SM.sensitivity_plots(indices, 'S1')

In the plot below, we see a heatmap of the $S_2$ sensitivity indices for pairs of the input variables.

In [ ]:
SM.sensitivity_plots(indices, 'S2')

### Analysis of the results for sampling over the whole range of allowed values.

As we can see from the S1 plot above, it appears that TMAX and SRAD play critical roles in determining the uncertainty of the output here.
We can inspect the source code to try and determine why these two variables play such a significant role.
Below is a code-block from the PETPT Fortran source code:

```fortran
      SLANG = SRAD*23.923
      EEQ = SLANG*(2.04E-4-1.83E-4*ALBEDO)*(TD+29.0)
      EO = EEQ*1.1

      IF (TMAX .GT. 35.0) THEN
        EO = EEQ*((TMAX-35.0)*0.05+1.1)
      ELSE IF (TMAX .LT. 5.0) THEN
        EO = EEQ*0.01*EXP(0.18*(TMAX+20.0))
      ENDIF

      EO = MAX(EO,0.0001)
```

In this code-block we can see that TMAX plays a critical role at the end of the PETPT function - that of dividing the function into three piecewise components based upon the value of TMAX.
AutoMATES can detect this using static code analysis on the conditional variables included in the model code.

Using this along with text-reading knowledge of the parameter bounds for TMAX,
we can divide the total range of values for TMAX into three separate bound ranges that we can use for further analysis:

1. -10.0ºC -- 5.0ºC
2. 5.0ºC -- 35.0ºC
3. 35.0ºC -- 55.0ºC

Below we show the S1/S2 results for these three different bound ranges.

### Bound range 1: $\mathrm{TMAX} \in [5, 35)$ 

In [ ]:
# Choosing the bound set
SM.bounds["tmax"] = [5.0, 34.99]
indices = SM.generate_indices()
SM.sensitivity_plots(indices, "S1")

In [ ]:
SM.sensitivity_plots(indices, "S2")

### Result set 1
Notice for this first result set that we are working with non-extreme values of TMAX. When the values for TMAX are no longer extreme, we see that it drops off significantly from the S1 indices and allows other variables to show how much more of an important role they play in determining model output.

#### Key insights
- SRAD now dominates all other uncertainty sources in the S1 indices
- The interaction between SRAD and XHLAI is now shown to be an important S2 indice

### Bound range 2: $\mathrm{TMAX} \in [-10, 5)$ 

In [ ]:
SM.bounds["tmax"] = [-10.0, 4.99]

In [ ]:
indices = SM.generate_indices()

In [ ]:
SM.sensitivity_plots(indices, "S1")

In [ ]:
SM.sensitivity_plots(indices, "S2")

### Result set 2
For this result set we are looking at the extreme lower end of values for TMAX. Here we see that TMAX is an even greater contributor to S1 sensitivity than over the whole range. This is likely due to the scaling of EO done by an exponential term involving TMAX in this portion of the piecewise function.

### Bound range 3: $\mathrm{TMAX} \in [35, 55]$ 

In [ ]:
SM.bounds["tmax"] = [35, 55]
indices = SM.generate_indices()

In [ ]:
SM.sensitivity_plots(indices, 'S1')

In [ ]:
SM.sensitivity_plots(indices, 'S2')

### Result set 3
Here we are testing the extreme high end of values for TMAX. For this bound range we are now observing:
- SRAD is the largest contributor to the S1 sensitivity
- TMAX is still a large contributor to the S1 sensitivity
- The total S1 sensitivity has increased dramatically, causing a large decrease in S2 total sensitivity which lowers the impact of the XHLAI/MSALB interaction term. This is likely due to the scaling of EO by TMAX in this branch of the piecewise function

## Take-home Message

In this demo we have shown how an initial sensitivity analysis of a scientific model can inform which variables to search for when doing single-variable range analysis.
AutoMATES enables us to search the source code of the model automatically to determine where in the code variables with high S1 sensitivities cause functional differences.
Using the parameter estimation from the TR component of AutoMATES and static progam analysis from the PA component of AutoMATES we can then perform a second set of sensitivity analysis upon a broken up range of the variable in question.
This allows AutoMATES to deliver deeper insights about the sensitivity of a scientific model to its inputs without requiring a modeler to:

- manually trek through the model code searching for the variable interactions of highest interest,
- find correct bound parameters for variables of interest in the literature, and
- create new sensitivity analysis experiments as appropriate for the new ranges of a variable of interest